# Step 1 - Get all contacts and campaigns
## One more time with feeling
#### All Contact/Maple Syrup Campaigns

In [ ]:
with
campaigns as (
    select 
        id as campaign_id,
        name as campaign_name,
        'Yes' as is_maple_syrup,
        start_date as campaign_start_date,
        end_date as campaign_end_date
    from staging.salesforce.stg_salesforce__campaign
    -- doesn't support (?i) syntax :(
    where name rlike '^[0-9]{4}\\s+.*\\b[Mm][Aa][Pp][Ll][Ee]\\b.*\\b[Ss][Yy][Rr][Uu][Pp]\\b.*$'
      and is_deleted=false
),

members as (
    select 
        campaign_id,
        contact_id        
    from staging.salesforce.stg_salesforce__campaign_member
    where is_deleted=false
),

full_campaigns as (
    select 
        c.*,
        m.contact_id
    from campaigns c
    inner join members m 
        on m.campaign_id=c.campaign_id
),

contacts as (
    select 
        id || account_id as contact_key,
        id as contact_id,
        account_id as contact_account_id,
        name as contact_name,
        contact_status_c as contact_status,
        contact_owner_id 
    from staging.intermediate.int_salesforce__contact
    where is_deleted=false
),

contacts_final as (
    select 
        c.contact_key as contact_account_key,
        c.contact_key as contact_referred_account_key,
        c.contact_id,
        c.contact_name,
        c.contact_account_id,
        c.contact_status,
        c.contact_owner_id,
        f.campaign_id,
        f.campaign_name,
        f.campaign_start_date,
        f.campaign_end_date
    from contacts c 
    left join full_campaigns f
        on c.contact_id=f.contact_id
)

select * from contacts_final

# Step 2 - Get all Portfolio accounts, their Opportunities and their referring accounts
## For Accounts, get create dates and referred by data 
## For the opps, get their current stage and create date and referred by data

In [ ]:
with
record_types as (
    select * from staging.salesforce.stg_salesforce__record_type
    where is_active=true
),

accounts as (
    select * from staging.intermediate.int_salesforce__account
    where is_deleted=false
),

contacts as (
    select * from staging.intermediate.int_salesforce__contact
    where is_deleted=false
),

opportunities as (
    select * from staging.intermediate.int_salesforce__opportunity
    where is_deleted=false
),

opportunity_histories as (
    select * from staging.salesforce.stg_salesforce__opportunity_history
    where is_deleted=false
),

get_portforlio_accounts as (
    select 
        a.id as portfolio_account_id,
        a.name as portfolio_account_name,
        cast(a.created_date as date) as portfolio_account_created_date,
        to_char(a.created_date, 'YYYY-MM') as portfolio_account_created_month,
        to_char(a.created_date, 'YYYY') as portfolio_account_created_year,
        case
            when a.referring_entity_c is null and c.account_id is null 
            then null
            else coalesce(a.referring_entity_c, c.account_id, 'XXXXXXXXXXXXXXXXXX') 
        end as portfolio_account_referring_account_id,
        a.referring_entity_contact_c as portfolio_account_referring_contact_id,
        r.name as portfolio_account_record_type
    from accounts a
    left join contacts c on c.id=a.referring_entity_contact_c
    left join record_types r on r.id=a.record_type_id and r.sobject_type='Account' and r.is_active=true
    -- where a.record_type_id in (
    --     select rt.id 
    --     from record_types rt
    --     where rt.sobject_type='Account'
    --       and rt.name='Portfolio Company'
    --   )
      --and a.referring_entity_c is null and a.referring_entity_contact_c is not null
),

get_all_accounts as (
    select 
        a.id as account_id,
        a.name as account_name,
        cast(a.created_date as date) as account_created_date,
        to_char(a.created_date, 'YYYY-MM') as account_created_month,
        to_char(a.created_date, 'YYYY') as account_created_year,
        r.name as account_record_type
    from accounts a
    left join record_types r on r.id=a.record_type_id and r.sobject_type='Account' and r.is_active=true
),

opportunity_stages as (
    select
        opportunity_id,
        stage_name,
        date(created_date) as stage_created_date,
        row_number() over (
            partition by opportunity_id
            order by created_date desc
        ) as rn
    from opportunity_histories
),

opportunity_current_stage as (
    select 
        opportunity_id,
        stage_name as current_stage,
        stage_created_date
    from opportunity_stages
    where rn=1
),

opportunity_columns as (
    select
        o.id as opportunity_id,
        o.name as opportunity_name,
        o.account_id as opportunity_account_id,
        s.current_stage as opportunity_current_stage,
        o.contact_id as opportunity_contact_id,
        o.referral_contact_c as opportunity_referring_contact_id,
        case
            when o.referral_source_c is null and c.account_id is null 
            then null 
            else coalesce(o.referral_source_c, c.account_id, 'XXXXXXXXXXXXXXXXXX') 
        end as opportunity_referring_account_id,
        date(o.created_date) as opportunity_created_date,
        to_varchar(o.created_date, 'YYYY-MM') as opportunity_created_month,
        to_varchar(o.created_date, 'YYYY') as opportunity_created_year,
        date(s.stage_created_date) as opportunity_stage_created_date,
        to_varchar(s.stage_created_date, 'YYYY-MM') as opportunity_stage_created_month,
        to_varchar(s.stage_created_date, 'YYYY') as opportunity_stage_created_year
    from opportunities o
    left join opportunity_current_stage s on s.opportunity_id=o.id
    left join contacts c on c.id=o.referral_contact_c
),

portfolio_accounts_opportunities as (
    select
        a.portfolio_account_id,
        a.portfolio_account_name,
        a.portfolio_account_created_date,
        a.portfolio_account_created_month,
        a.portfolio_account_created_year,
        a.portfolio_account_referring_account_id,
        a.portfolio_account_referring_contact_id,
        a.portfolio_account_record_type,
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_account_id,
        o.opportunity_current_stage,
        o.opportunity_contact_id,
        o.opportunity_referring_contact_id,
        o.opportunity_referring_account_id,
        o.opportunity_created_date,
        o.opportunity_created_month,
        o.opportunity_created_year,
        o.opportunity_stage_created_date,
        o.opportunity_stage_created_month,
        o.opportunity_stage_created_year
    from get_portforlio_accounts a
    left join opportunity_columns o on o.opportunity_account_id=a.portfolio_account_id
),

final as (
    select 
        a.account_id,
        a.account_name,
        p.portfolio_account_id,
        p.portfolio_account_name,
        p.opportunity_account_id,
        p.opportunity_id,
        p.opportunity_name,
        p.portfolio_account_referring_account_id,
        p.portfolio_account_referring_contact_id,
        p.opportunity_referring_contact_id,
        p.opportunity_referring_account_id,
        a.account_created_date,
        a.account_created_month,
        a.account_created_year,
        a.account_record_type,
        p.portfolio_account_created_date,
        p.portfolio_account_created_month,
        p.portfolio_account_created_year,
        p.portfolio_account_record_type,
        p.opportunity_current_stage,
        p.opportunity_contact_id,
        p.opportunity_created_date,
        p.opportunity_created_month,
        p.opportunity_created_year,
        p.opportunity_stage_created_date,
        p.opportunity_stage_created_month, 
        p.opportunity_stage_created_year
    from get_all_accounts a
    left join portfolio_accounts_opportunities p on a.account_id=p.portfolio_account_id
)

select count(distinct opportunity_id) from final
--order by opportunity_name
--where opportunity_id is not null
-- where portfolio_account_referring_account_id is not null
--   and account_id='0010W00002PsiN1QAJ'

-- now join the 